In [2]:
import sys
sys.version

'3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:43:09) [GCC 12.3.0]'

In [3]:
%pip install splink
%pip install pyodbc
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.blocking_rule_library import block_on
from splink.datasets import splink_datasets

df = splink_datasets.fake_1000

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "comparisons": [
        ctl.name_comparison("first_name"),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],
}

linker = DuckDBLinker(df, settings)
linker.estimate_u_using_random_sampling(max_pairs=1e6)

blocking_rule_for_training = block_on(["first_name", "surname"])

linker.estimate_parameters_using_expectation_maximisation(blocking_rule_for_training, estimate_without_term_frequencies=True)

blocking_rule_for_training = block_on("substr(dob, 1, 4)")  # block on year
linker.estimate_parameters_using_expectation_maximisation(blocking_rule_for_training, estimate_without_term_frequencies=True)


pairwise_predictions = linker.predict()

clusters = linker.cluster_pairwise_predictions_at_threshold(pairwise_predictions, 0.95)
clusters.as_pandas_dataframe(limit=5)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Iteration 1: Largest change in params was -0.491 in the m_probability of dob, level `Exact match`
Iteration 2: Largest change in params was 0.119 in probability_two_random_records_match
Iteration 3: Largest change in 

,cluster_id,unique_id,first_name,surname,dob,city,email,cluster,__splink_salt,tf_city
0,0,0,Robert,Alan,1971-06-24,None,robert255@smith.net,0,0.805617,NaN
1,0,1,Robert,Allen,1971-05-24,None,roberta25@smith.net,0,0.030378,NaN
2,0,2,Rob,Allen,1971-06-24,London,roberta25@smith.net,0,0.632551,0.212792
3,0,3,Robert,Alen,1971-06-24,Lonon,None,0,0.663057,0.007380
4,4,4,Grace,None,1997-04-26,Hull,grace.kelly52@jones.com,1,0.142784,0.001230


In [5]:
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.blocking_rule_library import block_on
from splink.datasets import splink_datasets
import logging, sys
logging.disable(sys.maxsize)

df = splink_datasets.fake_1000

settings = {
"link_type": "dedupe_only",
"blocking_rules_to_generate_predictions": [
block_on("first_name"),
block_on("surname"),
block_on("email")
]
}

linker = DuckDBLinker(df, settings)

linker.cumulative_num_comparisons_from_blocking_rules_chart()

alt.Chart(...)

In [6]:
import pyodbc
import os
import pandas as pd
import re

In [7]:
os.environ['DW_SERVER'] = "pdsqlirisdw01.hosted.lac.com"
os.environ['DW_DB'] = "PRD_APHIM_UODS_DW"
os.environ['DW_USER'] = "zz_dssteam"
os.environ['DW_PASS'] = "$3rv1se0neF1ve"

In [8]:
# Set up the connection
sqdw_con = pyodbc.connect(
    driver='sqlserver', 
    server=os.getenv('DW_SERVER'),
    database=os.getenv('DW_DB'),
    uid=os.getenv('DW_USER'),
    pwd=os.getenv('DW_PASS'),
    TrustServerCertificate='YES',
    timeout=10
)

# Execute a query
query = "SELECT top 10000 * FROM Fact.VW_Person WITH (NOLOCK) where PER_StreetAddress is not NULL"
dw_person = pd.read_sql(query, sqdw_con)
dw_person

/tmp/ipykernel_27875/882181193.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dw_person = pd.read_sql(query, sqdw_con)


,SynchronizationTimestamp,Status,PER_SiteCode,PER_RowID,PER_RootID,PER_ClientID,PER_CreateDate,PER_UpdateDate,PER_IsPatient,PER_IsContact,...,PER_MaritalStatusDR,PER_ResidenceCountyDR,PER_CountryofResidenceDR,PER_CountryofBirthDR,PER_PrimaryNationalityDR,PER_DeceasedStatusDR,PER_StatusFlagDR,PER_WorkOrSchoolLocationDR,PER_OccupationSettingTypeDR,PER_OccupationDR
0,2023-03-27 13:20:14.570,True,10100,679005,679005,438795,2000-07-06 00:00:00.000,2000-07-06,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
1,2023-03-27 13:20:14.570,True,10100,679018,679013,438803,2000-07-06 00:00:00.000,NaT,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
2,2023-03-27 13:20:14.570,True,10100,679031,679031,438820,2000-07-06 00:00:00.000,2000-07-06,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
3,2023-03-27 13:20:14.570,True,10100,679037,679037,438826,2000-04-27 00:00:00.000,2000-04-27,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
4,2023-03-27 13:20:14.570,True,10100,679047,679047,438836,2000-07-06 00:00:00.000,2000-07-06,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2023-03-27 13:20:52.220,True,10100,698816,698816,457565,2000-04-27 00:00:00.000,2000-04-27,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
9996,2023-03-27 13:20:52.220,True,10100,698819,698819,457568,2000-04-27 00:00:00.000,2000-04-27,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
9997,2023-03-27 13:20:52.220,True,10100,698825,698825,457573,2000-09-07 00:00:00.000,2000-09-07,True,False,...,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN
9998,2023-03-27 13:20:52.220,True,10100,698848,698848,457595,2021-04-14 07:14:33.180,2000-04-27,True,False,...,NaN,182151.0,0.0,0.0,None,None,None,None,NaN,NaN


In [9]:
dw_person["id"] = dw_person.index + 1
dw_person_subset=dw_person[["PER_StreetAddress","PER_Apartment","PER_City","PER_State","PER_Zip"]]
#dw_person_subset_2=dw_person_subset.rename(columns={"PER_StreetAddress": "original_address", "PER_Apartment": "original_apartment", "PER_City": "original_city","PER_State": "original_state","PER_Zip": "original_zip"})
#dw_person_subset_2
input_df = dw_person_subset
#input_df = dw_person

#input_df2=input_df.copy() #making a copy so we can compare approaches  

input_df["id"] = input_df.index + 1

input_df_test=input_df.copy(deep=True) #making a copy so we can compare approaches  

input_df

/tmp/ipykernel_27875/1746214022.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["id"] = input_df.index + 1


,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id
0,25458 Via Jardin,,SANTA CLARITA,CA,91355,1
1,13463 Mulberry Dr Apt 15,15,WHITTIER,CA,90605,2
2,3025 Hollycrest Dr,,HOLLYWOOD,CA,90068,3
3,681 Shatto Pl Apt 102,,LOS ANGELES,CA,90005,4
4,946 N N Mariposa Ave Apt 325,325,LOS ANGELES,CA,90029,5
...,...,...,...,...,...,...
9995,4445 Tujunga Ave # 2,,NORTH HOLLYWOOD,CA,91602,9996
9996,4377 New York Ave,,LOS ANGELES,CA,90022,9997
9997,2148 Clinton St,,LOS ANGELES,CA,90026,9998
9998,8978 Madison Ave,Apt a,South Gate,CA,90280,9999


In [10]:
dw_person.columns

Index(['SynchronizationTimestamp', 'Status', 'PER_SiteCode', 'PER_RowID',
       'PER_RootID', 'PER_ClientID', 'PER_CreateDate', 'PER_UpdateDate',
       'PER_IsPatient', 'PER_IsContact', 'PER_IsFamilyMember', 'PER_IsPrimary',
       'PER_IsVersion', 'PER_LastName', 'PER_FirstName', 'PER_MiddleName',
       'PER_ThirdName', 'PER_FourthName', 'PER_NameSuffix', 'PER_NamePrefix',
       'PER_SSN', 'PER_DOB', 'PER_PrimaryLanguage', 'PER_Age', 'PER_Ethnicity',
       'PER_ReportedRace', 'PER_DateOfUSArrival', 'PER_Sex',
       'PER_MaritalStatus', 'PER_StateNumber', 'PER_ResidenceCounty',
       'PER_CountryofResidence', 'PER_CountryofBirth',
       'PER_PrimaryNationality', 'PER_ParentOrGuardianName',
       'PER_LastNameAlphaUp', 'PER_FirstNameAlphaUp', 'PER_DOD',
       'PER_DeceasedStatus', 'PER_StatusFlag', 'PER_StreetAddress',
       'PER_Apartment', 'PER_City', 'PER_State', 'PER_Zip', 'PER_CensusTract',
       'PER_Latitude', 'PER_Longitude', 'PER_AddressStandardized',
       'PER_Co

In [11]:
import time

In [12]:
def clean_addr(df, address_var, apt_var, city_var, state_var, zip_var):
    # Replace backslashes with forward slashes in address
    df[address_var] = df[address_var].str.replace(r"\\", "/", regex=True)

    # Extract fractions like '1/2' or '3/4' from the address
    df['addr_frac'] = df[address_var].str.extract(r"(1/2|3/4)", expand=False)
    df[address_var] = df[address_var].str.replace(r"1/2|3/4", "", regex=True).str.strip()

    # Remove unnecessary punctuation and convert to uppercase
    df[address_var] = df[address_var].str.replace(r"[^a-zA-Z0-9\s,]", " ", regex=True).str.upper().str.strip()
    df[apt_var] = df[apt_var].str.replace(r"[^a-zA-Z0-9\s#\-/]", " ", regex=True).str.upper().str.strip()

    # Remove entirely blank entries
    for var in [address_var, apt_var, city_var, state_var, zip_var]:
        df[var] = df[var].replace(r"^\s*$", "", regex=True)

    # Simple regex to extract street (assuming 'number streetname' format)
    street_regex = r"^\d+\s[\w\s]+"
    df['addr'] = df[address_var].apply(lambda x: re.match(street_regex, x).group() if re.match(street_regex, x) else '')

    # Assuming city, state, and zip are already separated correctly
    df['city'] = df[city_var]
    df['state'] = df[state_var]
    df['zip'] = df[zip_var]

    return df

In [13]:
# Now apply the clean_addr function to the input DataFrame
cleaned_df1=clean_addr(df=input_df,
                        address_var="PER_StreetAddress",
                        apt_var="PER_Apartment", 
                        city_var="PER_City", 
                        state_var="PER_State",
                        zip_var="PER_Zip")

# cleaned_df now contains the cleaned address data
cleaned_df1

/tmp/ipykernel_27875/2848672108.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[address_var] = df[address_var].str.replace(r"\\", "/", regex=True)
/tmp/ipykernel_27875/2848672108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['addr_frac'] = df[address_var].str.extract(r"(1/2|3/4)", expand=False)
/tmp/ipykernel_27875/2848672108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip
0,25458 VIA JARDIN,,SANTA CLARITA,CA,91355,1,NaN,25458 VIA JARDIN,SANTA CLARITA,CA,91355
1,13463 MULBERRY DR APT 15,15,WHITTIER,CA,90605,2,NaN,13463 MULBERRY DR APT 15,WHITTIER,CA,90605
2,3025 HOLLYCREST DR,,HOLLYWOOD,CA,90068,3,NaN,3025 HOLLYCREST DR,HOLLYWOOD,CA,90068
3,681 SHATTO PL APT 102,,LOS ANGELES,CA,90005,4,NaN,681 SHATTO PL APT 102,LOS ANGELES,CA,90005
4,946 N N MARIPOSA AVE APT 325,325,LOS ANGELES,CA,90029,5,NaN,946 N N MARIPOSA AVE APT 325,LOS ANGELES,CA,90029
...,...,...,...,...,...,...,...,...,...,...,...
9995,4445 TUJUNGA AVE 2,,NORTH HOLLYWOOD,CA,91602,9996,NaN,4445 TUJUNGA AVE 2,NORTH HOLLYWOOD,CA,91602
9996,4377 NEW YORK AVE,,LOS ANGELES,CA,90022,9997,NaN,4377 NEW YORK AVE,LOS ANGELES,CA,90022
9997,2148 CLINTON ST,,LOS ANGELES,CA,90026,9998,NaN,2148 CLINTON ST,LOS ANGELES,CA,90026
9998,8978 MADISON AVE,APT A,South Gate,CA,90280,9999,NaN,8978 MADISON AVE,South Gate,CA,90280


In [12]:
# Define the keyword lists
invalid_addr1 = "|".join(["UNKNO", "HOMELES", "NEED ADDR", "NEED INFO", "HOMESLESS", "TRANSIENT", "SUPPRESS", "ENCAMPM", 
                          "UNDOMIC", "UNDERPASS", "HOMELESS", "NOADDRESS", "NO ADDRESS", "NONE PROVIDED", "REFUSED", 
                          "123 DEFAULT ST", "PT STATES NONE", "999 TRANSIENT WAY", "UNSHELTERED", "UNHOUSED", "UNKKNOWN", 
                          "UNKNWOWN", "UNKONW", "UNKOWN", "UNSTABLE HOUSING", "XNEEDX", "XNEEDSX", "HOMELEXX", "X NEED X", 
                          "TRANSET", "MISSING", "MAILING ADDRESS", "MAILING ONLY", "NOT AVAILABLE", "NOT PROVIDED", 
                          "THIS IS NOT A HOME ADDRESS", "MAIL RTND BAD ADDRESS", "MAILING ADDRESS ONLY", "NEED NEW ADDRESS", 
                          "FILLING OUT FORMS", "ADDRS VERF", "NO ACTUAL ADDRESS", "NEEDS ADDRESS", "DOES NOT HAVE AN ADDRESS", 
                          "REQUEST CALL", "UNABLE TO PROVIDE", "RETURN MAIL", "BAD ADDRESS", "UNABLE TO LOCATE", 
                          "NOT GIVEN", "NONE GIVEN", "NO KNOWN ADDRESS"])
peh_keywords = invalid_addr1 + "|450 BAUCHET"
peh_keywords1 = ['NOT PROVIDED', 'UNSPECIFIED', '*NONE', 'REFUSED', '`', 'NULL', 'NONE', 'NA', 'N/A',  
                 'ODR', 'SHELTER', 'IN TRANSIT', 'UNK']
peh_keywords2 = peh_keywords1 + ['CA', 'NOT CALIFORNIA', 'LAC']

# Continue defining the keyword lists
invalid_addr15 = "|UPDATE DEMO MAIL RTN|123 TEST|TEST PATIENT|TEST STREET|UPDATE DEMO|ROUTINE POSTPARTUM|" + \
                "123 MISSING|^HOMELES.*|^UNHOUSED.*|^UNSHELTER.*|^TRANSIENT.*|^UNK.*|^PRIV.*|^UNSPEC.*|^PT STATES.*|"
invalid_addr3 = peh_keywords1 + ['YES', '1234 ST', 'TBA', 'TBD', 'TEST 1', 'TEST', 'UNITED STATES', 'US', 'USA',
                                 'MAIL ONLY', 'MAIL RETURNED', 'MAILING', 'ANTHEM BLUE CROSS', 'SANTA MONICA BEACH',
                                 'SPRING', 'TEXT', 'NAME', '1 HOME', 'HOTEL', '1 FAKE', 'SPANISH', 
                                 'SPECIMEN NOT COLLECTED',  'UPDATE', '1 NEED', 'MAIN HOUSE', 'MANAGER', 
                                 'MANAGER OFFICE', 'MANAGER UNIT', 'XX', 'XX OTHER', 'PATIENT REFUSED', 
                                 'NOT PROVIDED', "^REFUS.*", "^NO ADDRESS.*", 'NOT AVAILABLE']
invalid_addr4 = invalid_addr3 + ['CALIFORNIA', 'CA']
invalid_addr5 = invalid_addr4 + ['LOS ANGELES', 'LA', '# LA', '#LA', 'N A', 'NA', '999 TRANSIENT WAY', 
                                 '999 TRANSIENT', '99 JAIL CASE', '123 UNK', '9999 TRANSIENT', '9999 TRANSIENT WAY', 
                                 '9999 TRANSIENT', 'SOUTHBAY ER', 'XXX HOMELESS XXX', 'POST OFFICE BOX', 
                                 'PTIENT STATES', 'SUNSET ED', 'VALLEY PALMS CARE CENTER', 'W LOS ANGELES ED', 
                                 'WLA ED', 'UNK']

# Invalid apartment numbers
invalid_apt1 = invalid_addr1 + "|UNITED CARE FACILITES|HEALTH CARE|WE DO HAVE A NEW REGISTRATION|" + \
               "UNITED CEREBRAL PALSEY|UNITED STATES"
invalid_apt2 = invalid_addr4 + ['YES', '-', '1 LAB', 'APT SUITE FLOOR ETC.', 'NO APT']
invalid_apt3 = invalid_addr5 + ["UNITED CARE FACILITES", "HEALTH CARE", "WE DO HAVE A NEW REGISTRATION", 
                                "UNITED CEREBRAL PALSEY", "UNITED STATES", "BELLFLOWER CHRISTIAN", 
                                "GRAND PARK CONVALESCENT", 'BRUIN PLAZA', 'YES', '-', '1 LAB', 
                                'APT SUITE FLOOR ETC', 'NO APT']

# Key phrases for invalid apartment units
invalid_apt_frag = ["CONV", "HEALTH", "MEDICAL", "NURSING", "ACUTE", "CARE", "SOBER", "REHAB", "ASSISTED",
                    "LIVING", "MANOR", "VILLA", "RETIREMENT", "BEH", "SENIOR", "SNF", "RESID", "SANIT", "CHURCH",
                    "ACUTE", "HEALTHCARE", "WELLNESS", "CONGREGATE", "CONVALESCENT", "GLENDORA GRAND", "REHABILITATION",
                    # Other terms
                    "HOME", "HOUSE", 'HEIGHT', 'COUNTRY', 'GARDEN', 'COTTAGE',  'ADDR', 'C/O', 
                    'POBOX', 'PLAZA', 'UNABLE', 'TRANSIENT', 'JAIL', 'PRISON', 'THE', 'SAME', 'VISTA', 'MAIL', 'PARK', 
                    'OFFICE', 'LAPD', 'LETTER', 'HOTEL', 'INN', 'MOTEL', 'COUNTY', 'UPPER', 'LOWER', 'REAR', 'DOWN', 
                    'UP', 'TEST', 'UNI', 'STADIUM']

# Invalid cities
invalid_city = ['NULL', 'NONE', 'NA', 'N/A', 'YES', 'REFUSED', '`', 'NOT PROVIDED', 'UNK', '', 'UNKKNOWN', 
                'UNKNWOWN', 'UNKONW', 'UNKOWN', 'UNSPECIFIED', 'UNITED STATES', 'US', 'CALIFORNIA', 'CA', ' ', '*NONE']

# POBOX keywords
pobox_keywords = "POBOX|PO BOX|P0 BOX|POSTAL|PO BX|9O BOX|OK BOX|P 0  BOX|P BOX|PB BOX|PI BOX|PIC BOX|P.O. BOX|" + \
                 "P.O.BOX|PIO BOX|PITZER BOX|PO B0X|PO BAX|PO BOS|PO BOT|PO BOV|POX BOX|SHOW BOX|POP BOX|P O BOX|POX BOX"

# Unknown phone numbers
unknown_numbers = ["0000000000", "1111111111", "2222222222", "3333333333", "4444444444", "5555555555", 
                   "6666666666", "7777777777", "8888888888", "9999999999", "0000000001"]

# Unknown email keywords
unknown_emails1 = "GENERATED|NOEMAIL|NOMAIL|REFUSE|UNKNOWN|NO@NO|NA@NA|PATIENT|" + \
                  "DOES NOT|MISSING|NONAME|DECLINED|ASK FOR|NOT GIVEN"
unknown_emails2 = ['NAGMAIL.COM', 'N', 'DECLINED', 'NA', 'NO', 'TEST', 'NONE', 'GOBACKTOASK']

# Unknown names
unknown_names1 = "NO FIRST NAME|NO INFO"
unknown_names2 = ["TEST", "PATIENT"]

In [13]:
%pip install usaddress
import usaddress

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Assuming invalid_addr1, invalid_addr2, ..., invalid_apt1, ..., etc. are lists
# Convert any string components to lists
invalid_addr1_list=invalid_addr1.split('|')
invalid_addr15_list=invalid_addr15.split('|')
pobox_keywords_list = pobox_keywords.split('|')
invalid_apt1_list = invalid_apt1.split('|')
unknown_emails1_list = unknown_emails1.split('|')
unknown_names1_list = unknown_names1.split('|')

# Combine all keywords into a single list
#all_invalid_keywords = invalid_addr1_list + invalid_addr15_list +invalid_addr3 + invalid_addr4 +\
#invalid_addr5 + invalid_apt1_list + invalid_apt2 + invalid_apt3 + pobox_keywords_list + unknown_numbers +\
#unknown_emails1_list +  unknown_emails2 + unknown_names1_list + unknown_names2

# Combine all keywords into a single list
# Ensure all keywords are strings and in uppercase for case-insensitive comparison
all_invalid_keywords = [str(keyword).upper() for keyword in ( invalid_addr1_list + invalid_addr15_list +invalid_addr3 + invalid_addr4 +\
invalid_addr5 + invalid_apt1_list + invalid_apt2 + invalid_apt3 + pobox_keywords_list + unknown_numbers +\
unknown_emails1_list +  unknown_emails2 + unknown_names1_list + unknown_names2)]

def check_invalid_component(parsed_addr, invalid_keywords):
    for component_type, component_value in parsed_addr.items():
        # Skip certain components from direct keyword matching
        if component_type in ['AddressNumber', 'AddressNumberPrefix', 'AddressNumberSuffix', 'StreetName', 'StreetNamePostType', 
                    'StreetNamePreDirectional','SubaddressType','StreetNamePostDirectional','OccupancyType','OccupancyIdentifier','PlaceName','StreetNamePreType']:
            continue

        if any(keyword in component_value.upper() for keyword in invalid_keywords):
            print(f"Invalid component found: {component_type} -> {component_value}")
            return True
    return False

def clean_address(address, invalid_keywords):
    try:
        parsed_address, _ = usaddress.tag(address)
        if check_invalid_component(parsed_address, invalid_keywords):
            return ""  # Return empty string if invalid
        return address  # Return original address if valid
    except usaddress.RepeatedLabelError as e:
        print(f"Error parsing address '{address}': {e}")
        return ""

input_df2 = cleaned_df1

# Apply the cleaning function to the 'addr' field
input_df['addr_valid'] = input_df2['addr'].apply(lambda x: clean_address(x, all_invalid_keywords))

#print(input_df2[['addr', 'addr_valid']])
input_df2

Invalid component found: StateName -> ST
Invalid component found: StateName -> ST
Invalid component found: StateName -> RD
Error parsing address '130 E AVENUE E SPC 429': 
ERROR: Unable to tag this string because more than one area of the string has the same label

ORIGINAL STRING:  130 E AVENUE E SPC 429
PARSED TOKENS:    [('130', 'AddressNumber'), ('E', 'StreetName'), ('AVENUE', 'StreetNamePostType'), ('E', 'StreetNamePostDirectional'), ('SPC', 'StreetNamePreType'), ('429', 'StreetName')]
UNCERTAIN LABEL:  StreetName

When this error is raised, it's likely that either (1) the string is not a valid person/corporation name or (2) some tokens were labeled incorrectly

To report an error in labeling a valid name, open an issue at https://github.com/datamade/usaddress/issues/new - it'll help us continue to improve probablepeople!

For more information, see the documentation at https://usaddress.readthedocs.io/
Error parsing address '719 1 4 S OSAGE AVE': 
ERROR: Unable to tag this string 

/tmp/ipykernel_15391/1978449062.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['addr_valid'] = input_df2['addr'].apply(lambda x: clean_address(x, all_invalid_keywords))


,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE


In [16]:
#Here we will do the same thing with the city, state and zipcodes i.e. compare them to the invalid keywords

# Define invalid keyword lists for city, state, and zipcode
invalid_city_keywords = invalid_addr4 # Add more as needed
invalid_state_keywords = invalid_addr3  # Add more as needed
invalid_zip_keywords = invalid_addr5  # Add more as needed

def check_invalid_component(parsed_addr, city_keywords, state_keywords, zip_keywords):
    is_invalid = False
    for component_type, component_value in parsed_addr.items():
        if component_type == 'PlaceName' and any(kw in component_value.upper() for kw in city_keywords):
            is_invalid = True
        elif component_type == 'StateName' and any(kw in component_value.upper() for kw in state_keywords):
            is_invalid = True
        elif component_type == 'ZipCode' and any(kw in component_value.upper() for kw in zip_keywords):  # Corrected key
            is_invalid = True
    return is_invalid

def clean_location(city, state, zipcode, city_keywords, state_keywords, zip_keywords):
    try:
        pseudo_address = f"{city}, {state} {zipcode}"  # Corrected variable name
        parsed_address, _ = usaddress.tag(pseudo_address)
        if check_invalid_component(parsed_address, city_keywords, state_keywords, zip_keywords):
            return "", "", ""  # Return empty strings if invalid
        return city, state, zipcode  # Return original values if valid
    except usaddress.RepeatedLabelError:
        print(f"Error parsing pseudo address '{city}, {state} {zipcode}'")
        return "", "", ""  # Handle parsing errors

# Sample DataFrame and applying the function (assuming input_df2 is defined)
input_df2[['city_valid', 'state_valid', 'zip_valid']] = input_df2.apply(
    lambda row: clean_location(row['city'], row['state'], row['zip'],
                               invalid_city_keywords, invalid_state_keywords, invalid_zip_keywords),
    axis=1, result_type='expand')

#print(input_df2[['city', 'city_valid', 'state', 'state_valid', 'zip', 'zip_valid']])
input_df2

/tmp/ipykernel_15391/3640814227.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df2[['city_valid', 'state_valid', 'zip_valid']] = input_df2.apply(
/tmp/ipykernel_15391/3640814227.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df2[['city_valid', 'state_valid', 'zip_valid']] = input_df2.apply(
/tmp/ipykernel_15391/3640814227.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406


In [15]:
#I need to also do the same thing for the apt field
# Define your list of invalid keywords for apartment numbers
invalid_apt_keywords = invalid_addr5  # Add your keywords

def is_valid_apt(apt, invalid_keywords):
    # Check if the apartment number is in the invalid keywords
    if any(kw.lower() in apt.lower() for kw in invalid_keywords):
        return ""
    return apt

# Apply the function to the 'PER_Apartment' field
input_df2['apt_valid'] = input_df2['PER_Apartment'].apply(lambda x: is_valid_apt(x, invalid_apt_keywords))

#print(input_df2[['city', 'state', 'zip', 'PER_Apartment', 'apt_valid']])

input_df2

/tmp/ipykernel_4657/1958128426.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df2['apt_valid'] = input_df2['PER_Apartment'].apply(lambda x: is_valid_apt(x, invalid_apt_keywords))


,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,6233 AUCKLAND AVE,,NORTH HOLLYWOOD,CA,91606,1,NaN,6233 AUCKLAND AVE,NORTH HOLLYWOOD,CA,91606,6233 AUCKLAND AVE,NORTH HOLLYWOOD,CA,91606,
1,216 S SOTO ST,,LOS ANGELES,CA,90033,2,NaN,216 S SOTO ST,LOS ANGELES,CA,90033,216 S SOTO ST,LOS ANGELES,CA,90033,
2,4085 W 138TH CT APT A,,INGLEWOOD,CA,90301,3,NaN,4085 W 138TH CT APT A,INGLEWOOD,CA,90301,4085 W 138TH CT APT A,INGLEWOOD,CA,90301,
3,245 N ALVARADO ST APT 302,,LOS ANGELES,CA,90026,4,NaN,245 N ALVARADO ST APT 302,LOS ANGELES,CA,90026,245 N ALVARADO ST APT 302,LOS ANGELES,CA,90026,
4,5002 W 135TH ST,,HAWTHORNE,CA,90250,5,NaN,5002 W 135TH ST,HAWTHORNE,CA,90250,5002 W 135TH ST,HAWTHORNE,CA,90250,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3800 BRADFORD ST SPC 115,,La Verne,CA,91750,9996,NaN,3800 BRADFORD ST SPC 115,La Verne,CA,91750,3800 BRADFORD ST SPC 115,La Verne,CA,91750,
9996,1626 WILCOX AVE APT 390,,LOS ANGELES,CA,90028,9997,NaN,1626 WILCOX AVE APT 390,LOS ANGELES,CA,90028,1626 WILCOX AVE APT 390,LOS ANGELES,CA,90028,
9997,3320 VETERAN AVE,,LOS ANGELES,CA,90034,9998,NaN,3320 VETERAN AVE,LOS ANGELES,CA,90034,3320 VETERAN AVE,LOS ANGELES,CA,90034,
9998,7010 SEPULVEDA BLVD APT 126,,VAN NUYS,CA,91405,9999,NaN,7010 SEPULVEDA BLVD APT 126,VAN NUYS,CA,91405,7010 SEPULVEDA BLVD APT 126,VAN NUYS,CA,91405,


In [17]:
apt_keywords = ['Apt', 'Unit', 'Suite', 'Bldg', 'Room', 'Fl', '#']

# Regular expression pattern for extracting apartment/unit numbers
apt_pattern = r'\b(?:' + '|'.join(apt_keywords) + r')\s*\S+'

def extract_apt(addr):
    match = re.search(apt_pattern, addr, re.IGNORECASE)
    return match.group(0) if match else ""

# Extract and populate the apt_valid field
input_df2['apt_valid'] = input_df2['addr_valid'].apply(extract_apt)

#print(input_df2[['addr_valid', 'apt_valid']])
input_df2

/tmp/ipykernel_15391/4187182795.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df2['apt_valid'] = input_df2['addr_valid'].apply(extract_apt)


,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260,
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717,APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706,
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406,


In [18]:
def clean_repeat_char(df, var, repeat_num=0):
    # Create a regular expression for repeated characters
    repeat_char_regex = r"^(.)\1*$"

    # Identify rows with invalid data
    df['invalid'] = df[var].apply(lambda x: (bool(re.match(repeat_char_regex, x)) and len(x) > repeat_num) or x in ["0", "00"])

    # Replace invalid data with an empty string
    df[var + '_new'] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Assuming cleaned_df2 is your DataFrame and it already has an 'apt_valid' column
# If not, you need to create it before applying the function
# For example:
# cleaned_df2['apt_valid'] = ...

cleaned_df3 = clean_repeat_char(input_df2, "addr_valid")
cleaned_df3 = clean_repeat_char(cleaned_df3, "apt_valid", repeat_num=3)
cleaned_df3 = clean_repeat_char(cleaned_df3, "city_valid")
cleaned_df3 = clean_repeat_char(cleaned_df3, "state_valid")
cleaned_df3 = clean_repeat_char(cleaned_df3, "zip_valid")

# Update original columns with the cleaned data
cleaned_df3['addr_valid'] = cleaned_df3['addr_valid_new']
cleaned_df3['apt_valid'] = cleaned_df3['apt_valid_new']
cleaned_df3['city_valid'] = cleaned_df3['city_valid_new']
cleaned_df3['state_valid'] = cleaned_df3['state_valid_new']
cleaned_df3['zip_valid'] = cleaned_df3['zip_valid_new']

# Drop the temporary columns
cleaned_df3.drop(columns=['invalid', 'addr_valid_new', 'apt_valid_new', 'city_valid_new', 'state_valid_new', 'zip_valid_new'], inplace=True)

cleaned_df3

/tmp/ipykernel_15391/423312547.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid'] = df[var].apply(lambda x: (bool(re.match(repeat_char_regex, x)) and len(x) > repeat_num) or x in ["0", "00"])
/tmp/ipykernel_15391/423312547.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[var + '_new'] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)
/tmp/ipykernel_15391/423312547.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260,
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717,APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706,
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406,


In [19]:
def clean_spec_char(df, var):
    # Define a regex pattern that checks for the absence of alphanumeric characters
    spec_char_regex = r"^[^a-zA-Z\d]+$"

    # Identify rows with only special characters and mark them as invalid
    df['invalid'] = df[var].apply(lambda x: bool(re.match(spec_char_regex, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df3 is your existing DataFrame
cleaned_df4 = clean_spec_char(cleaned_df3, "addr_valid")
cleaned_df4['addr_valid'] = cleaned_df4['new_addr_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "apt_valid")
cleaned_df4['apt_valid'] = cleaned_df4['new_apt_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "city_valid")
cleaned_df4['city_valid'] = cleaned_df4['new_city_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "state_valid")
cleaned_df4['state_valid'] = cleaned_df4['new_state_valid']
cleaned_df4 = clean_spec_char(cleaned_df4, "zip_valid")
cleaned_df4['zip_valid'] = cleaned_df4['new_zip_valid']

# Drop the temporary columns
cleaned_df4.drop(columns=['invalid', 'new_addr_valid', 'new_apt_valid', 'new_city_valid', 'new_state_valid', 'new_zip_valid'], inplace=True)

cleaned_df4

/tmp/ipykernel_15391/2729441260.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid'] = df[var].apply(lambda x: bool(re.match(spec_char_regex, x)) if pd.notna(x) else False)
/tmp/ipykernel_15391/2729441260.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)
/tmp/ipykernel_15391/2729441260.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260,
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717,APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706,
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406,


In [20]:
def clean_num_only(df, var):
    # Remove all non-alphanumeric characters and check if the remaining string is numeric
    df['invalid'] = df[var].apply(lambda x: x.isalnum() and x.isdigit() if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df4 is your existing DataFrame
cleaned_df5 = clean_num_only(cleaned_df4, "addr_valid")
cleaned_df5['addr_valid'] = cleaned_df5['new_addr_valid']
cleaned_df5 = clean_num_only(cleaned_df5, "city_valid")
cleaned_df5['city_valid'] = cleaned_df5['new_city_valid']
cleaned_df5 = clean_num_only(cleaned_df5, "state_valid")
cleaned_df5['state_valid'] = cleaned_df5['new_state_valid']

# Drop the temporary columns
cleaned_df5.drop(columns=['invalid', 'new_addr_valid', 'new_city_valid', 'new_state_valid'], inplace=True)
cleaned_df5

/tmp/ipykernel_15391/3839269998.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid'] = df[var].apply(lambda x: x.isalnum() and x.isdigit() if pd.notna(x) else False)
/tmp/ipykernel_15391/3839269998.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)
/tmp/ipykernel_15391/3839269998.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260,
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717,APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706,
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406,


In [21]:
#Not Running this now as we just looking at addresses
def remove_email(df, var):
    # Define a regex pattern for email addresses
    email_regex = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"

    # Identify rows with email addresses and mark them as invalid
    df['invalid'] = df[var].apply(lambda x: bool(re.search(email_regex, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Example usage
# Assume you have a DataFrame named 'your_df' with a column 'original_address'
# your_df = pd.DataFrame(...) # your actual DataFrame
cleaned_df6 = remove_email(cleaned_df5, "PER_Email")
#cleaned_df6['original_address'] = cleaned_df6['new_original_address']
#cleaned_df.drop(columns=['invalid', 'new_original_address'], inplace=True)

cleaned_df6

KeyError: 'PER_Email'

In [22]:
def clean_dates_only(df, var):
    # Define regex patterns for date formats
    date_patterns = [
        r"\d{2}/\d{2}/\d{4}",  # mm/dd/yyyy
        r"\d{2}/\d{2}/\d{2}",  # mm/dd/yy
        r"\d{2}-\d{2}-\d{4}",  # mm-dd-yyyy
        r"\d{2}-\d{2}-\d{2}"   # mm-dd-yy
    ]

    # Combine patterns into a single regex
    combined_pattern = '|'.join(date_patterns)

    # Identify rows with date-like strings
    df['invalid'] = df[var].apply(lambda x: bool(re.search(combined_pattern, x)) if pd.notna(x) else False)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df6 is your existing DataFrame
cleaned_df7 = clean_dates_only(cleaned_df5, "addr_valid") ## we only limited it to the address components need to use df5 instead of df6
cleaned_df7['addr_valid'] = cleaned_df7['new_addr_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "apt_valid")
cleaned_df7['apt_valid'] = cleaned_df7['new_apt_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "city_valid")
cleaned_df7['city_valid'] = cleaned_df7['new_city_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "state_valid")
cleaned_df7['state_valid'] = cleaned_df7['new_state_valid']
cleaned_df7 = clean_dates_only(cleaned_df7, "zip_valid")
cleaned_df7['zip_valid'] = cleaned_df7['new_zip_valid']

# Drop the temporary columns
cleaned_df7.drop(columns=['invalid', 'new_addr_valid', 'new_apt_valid', 'new_city_valid', 'new_state_valid', 'new_zip_valid'], inplace=True)

cleaned_df7

/tmp/ipykernel_15391/381045389.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid'] = df[var].apply(lambda x: bool(re.search(combined_pattern, x)) if pd.notna(x) else False)
/tmp/ipykernel_15391/381045389.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)
/tmp/ipykernel_15391/381045389.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260,
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717,APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706,
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406,


In [23]:
def clean_two_word(df, var):
    street_suffix = ["RD", "ST", "ROAD", "STREET", "PL", "PLACE", "LN", "LANE", "UPDATE", "OTHER"]
    
    def is_invalid(address):
        if pd.isna(address):
            return False
        words = re.sub(r"[^a-zA-Z0-9\s]", "", address.replace("  ", " ")).split()
        if len(words) != 2:
            return False
        return (words[1] in street_suffix and 
                bool(re.match(r"^(.)\1*$", words[0])) and 
                len(words[0]) > 1) or (words[0].replace("0", "") == "")

    df['invalid'] = df[var].apply(is_invalid)

    # Replace invalid data with an empty string
    df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)

    return df

# Apply the function to the DataFrame
# Assume cleaned_df7 is your existing DataFrame
cleaned_df8 = clean_two_word(cleaned_df7, "addr_valid")
cleaned_df8['addr_valid'] = cleaned_df8['new_addr_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "apt_valid")
cleaned_df8['apt_valid'] = cleaned_df8['new_apt_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "city_valid")
cleaned_df8['city_valid'] = cleaned_df8['new_city_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "state_valid")
cleaned_df8['state_valid'] = cleaned_df8['new_state_valid']
cleaned_df8 = clean_two_word(cleaned_df8, "zip_valid")
cleaned_df8['zip_valid'] = cleaned_df8['new_zip_valid']

# Drop the temporary columns
cleaned_df8.drop(columns=['invalid', 'new_addr_valid', 'new_apt_valid', 'new_city_valid', 'new_state_valid', 'new_zip_valid'], inplace=True)

cleaned_df8

/tmp/ipykernel_15391/3628750843.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid'] = df[var].apply(is_invalid)
/tmp/ipykernel_15391/3628750843.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_' + var] = df.apply(lambda row: "" if row['invalid'] else row[var], axis=1)
/tmp/ipykernel_15391/3628750843.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,PER_StreetAddress,PER_Apartment,PER_City,PER_State,PER_Zip,id,addr_frac,addr,city,state,zip,addr_valid,city_valid,state_valid,zip_valid,apt_valid
0,PO BOX 1169,,Avalon,CA,90704,1,NaN,,Avalon,CA,90704,,Avalon,CA,90704,
1,14410 FIRMONA AVE,,Lawndale,CA,90260,2,NaN,14410 FIRMONA AVE,Lawndale,CA,90260,14410 FIRMONA AVE,Lawndale,CA,90260,
2,15111 FREEMAN AVE UNIT 92,,Lawndale,CA,90260,3,NaN,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,15111 FREEMAN AVE UNIT 92,Lawndale,CA,90260,UNIT 92
3,25843 NARBONNE AVE APT 3,,Lomita,CA,90717,4,NaN,25843 NARBONNE AVE APT 3,Lomita,CA,90717,25843 NARBONNE AVE APT 3,Lomita,CA,90717,APT 3
4,1160 KINBRAE AVE,UNITED CARE HOMES,Hacienda Heights,CA,91745,5,NaN,1160 KINBRAE AVE,Hacienda Heights,CA,91745,1160 KINBRAE AVE,Hacienda Heights,CA,91745,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9871 LINDEN ST,,Bellflower,CA,90706,9996,NaN,9871 LINDEN ST,Bellflower,CA,90706,9871 LINDEN ST,Bellflower,CA,90706,
9996,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9997,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9997,628 HUNTLEY DR APT 4,,West Hollywood,CA,90069,9998,NaN,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,628 HUNTLEY DR APT 4,West Hollywood,CA,90069,APT 4
9998,7277 VALJEAN AVE,,Van Nuys,CA,91406,9999,NaN,7277 VALJEAN AVE,Van Nuys,CA,91406,7277 VALJEAN AVE,Van Nuys,CA,91406,


In [24]:
def unit_parse(df, var):
    # List of apartment keywords
    apt_names_st = ['APT', 'APP', 'AP', 'APACE', 'APARTMENT', 'CONDO', 'NO', 'STE', 'SUITE', 'SPC', 'UNIT', 
                    'BUILD', 'BLDG', 'BUILDING', 'RM', 'ROOM']

    # Create regex pattern for extracting unit
    apt_pattern_out = r"(?i)\b(" + "|".join(apt_names_st) + r")\s*\#?\s*[\d\w]+"

    # Function to extract unit
    def extract_unit(address):
        match = re.search(apt_pattern_out, address)
        return match.group(0) if match else ''

    # Function to remove unit
    def remove_unit(address):
        return re.sub(apt_pattern_out, '', address).strip()

    # Apply the functions to create new columns
    df['new_address'] = df[var].apply(remove_unit)
    df['new_address_unit'] = df[var].apply(extract_unit)

    return df


cleaned_df9 = unit_parse(cleaned_df8, "addr_valid")

# Example output
print(cleaned_df9)

              PER_StreetAddress      PER_Apartment          PER_City  \
0                   PO BOX 1169                               Avalon   
1             14410 FIRMONA AVE                             Lawndale   
2     15111 FREEMAN AVE UNIT 92                             Lawndale   
3      25843 NARBONNE AVE APT 3                               Lomita   
4              1160 KINBRAE AVE  UNITED CARE HOMES  Hacienda Heights   
...                         ...                ...               ...   
9995             9871 LINDEN ST                           Bellflower   
9996       628 HUNTLEY DR APT 4                       West Hollywood   
9997       628 HUNTLEY DR APT 4                       West Hollywood   
9998           7277 VALJEAN AVE                             Van Nuys   
9999            716 W RAMONA RD                             Alhambra   

     PER_State PER_Zip     id addr_frac                       addr  \
0           CA   90704      1       NaN                          

/tmp/ipykernel_15391/2930121224.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_address'] = df[var].apply(remove_unit)
/tmp/ipykernel_15391/2930121224.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['new_address_unit'] = df[var].apply(extract_unit)


In [23]:
cleaned_df9.columns

Index(['PER_StreetAddress', 'PER_Apartment', 'PER_City', 'PER_State',
       'PER_Zip', 'id', 'addr_frac', 'addr', 'city', 'state', 'zip',
       'addr_valid', 'city_valid', 'state_valid', 'zip_valid', 'apt_valid',
       'new_address', 'new_address_unit'],
      dtype='object')

In [26]:
print(cleaned_df9.count()[0])
print( len(df.columns))

10000
7


/tmp/ipykernel_15391/2850720894.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(cleaned_df9.count()[0])


In [27]:
print(dw_person.count()[0])
print( len(dw_person.columns))

10000
84


/tmp/ipykernel_15391/743542996.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(dw_person.count()[0])


In [28]:
merged_left = pd.merge(left=dw_person, right=cleaned_df9, how='left', left_on='id', right_on='id')

In [29]:
print(merged_left.count()[0])
print( len(merged_left.columns))

10000
101


/tmp/ipykernel_15391/681226224.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(merged_left.count()[0])


In [30]:
merged_left.columns

Index(['SynchronizationTimestamp', 'Status', 'PER_SiteCode', 'PER_RowID',
       'PER_RootID', 'PER_ClientID', 'PER_CreateDate', 'PER_UpdateDate',
       'PER_IsPatient', 'PER_IsContact',
       ...
       'city', 'state', 'zip', 'addr_valid', 'city_valid', 'state_valid',
       'zip_valid', 'apt_valid', 'new_address', 'new_address_unit'],
      dtype='object', length=101)

In [31]:
merged_left_subset=merged_left[['PER_LastName', 'PER_FirstName', 'PER_MiddleName','PER_ThirdName', 'PER_FourthName', 'PER_NameSuffix', 'PER_NamePrefix',
'PER_SSN', 'PER_DOB', 'PER_PrimaryLanguage', 'PER_Age', 'PER_Ethnicity','PER_Sex', 'city_valid', 'state_valid',
       'zip_valid', 'apt_valid', 'new_address', 'new_address_unit', 'id']]

In [32]:
counts = merged_left_subset.isna().sum()
print(counts.sort_values())

PER_LastName              0
new_address               0
apt_valid                 0
zip_valid                 0
state_valid               0
city_valid                0
PER_Sex                   0
new_address_unit          0
id                        0
PER_SSN                   0
PER_FirstName             0
PER_DOB                   2
PER_MiddleName           13
PER_NameSuffix           17
PER_Ethnicity           360
PER_NamePrefix         1370
PER_FourthName         1370
PER_ThirdName          1370
PER_PrimaryLanguage    9906
PER_Age                9995
dtype: int64


In [33]:
from splink.duckdb.linker import DuckDBLinker
#from splink.spark.linker import SparkLinker

settings = {
    "unique_id_column_name": "id",
    "link_type": "dedupe_only",
}

linker = DuckDBLinker(merged_left_subset, settings)

In [34]:
linker.missingness_chart()

alt.LayerChart(...)

In [35]:
linker.profile_columns(list(merged_left_subset.columns))

alt.VConcatChart(...)

In [1]:
from splink.duckdb.blocking_rule_library import block_on

blocking_rules = [
        block_on("PER_LastName"),
        block_on("new_address"),
        block_on("PER_MiddleName"),
        block_on("PER_FirstName"),
        block_on("PER_DOB"),
        block_on("zip_valid"),
        block_on("PER_SSN")
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

NameError: name 'linker' is not defined

In [39]:
from splink.duckdb.blocking_rule_library import block_on

blocking_rules = [
        block_on("PER_LastName"),
        block_on("new_address"),
        block_on("PER_FirstName"),
        block_on("PER_DOB"),
        block_on("zip_valid")
]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

alt.Chart(...)

In [35]:
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl


settings = {
    "unique_id_column_name": "id",
    "link_type": "link_and_dedupe",
    "blocking_rules_to_generate_predictions": blocking_rules,
    "comparisons": [
        ctl.name_comparison("PER_FirstName", term_frequency_adjustments=True),
        ctl.name_comparison("PER_LastName", term_frequency_adjustments=True),
        ctl.date_comparison("PER_DOB", 
                            damerau_levenshtein_thresholds=[],
                            cast_strings_to_date=True,
                            invalid_dates_as_null=True,
                            date_format="%Y%m%d"),
        cl.levenshtein_at_thresholds("PER_SSN", [2]),
        cl.exact_match("new_address", term_frequency_adjustments=True),
        cl.exact_match("zip_valid", term_frequency_adjustments=True),
    ],
    "retain_intermediate_calculation_columns": True,
}

linker = DuckDBLinker(merged_left_subset, settings)

In [36]:
deterministic_rules = [
    "l.PER_SSN = r.PER_SSN",
    "l.PER_FirstName = r.PER_FirstName and l.PER_LastName = r.PER_LastName and l.PER_DOB = r.PER_DOB",
    "l.PER_FirstName = r.PER_LastName and l.PER_LastName = r.PER_FirstName and l.PER_DOB = r.PER_DOB"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.9)

linker.estimate_u_using_random_sampling(max_pairs=1e6)

In [97]:
FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

NameError: name 'FloatProgress' is not defined

In [37]:
em_blocking_rule_1 = block_on("substr(PER_DOB,1,3)")
em_blocking_rule_2 = block_on("substr(zip_valid,1,2)")
session_dob = linker.estimate_parameters_using_expectation_maximisation(em_blocking_rule_1)
session_postcode = linker.estimate_parameters_using_expectation_maximisation(em_blocking_rule_2)

In [38]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [40]:
results = linker.predict(threshold_match_probability=0.2)

In [41]:
linker.roc_chart_from_labels_column("cluster")

SplinkException: Error executing the following sql for table `__splink__df_predict`(__splink__df_predict_6f9466d01):
CREATE TABLE __splink__df_predict_6f9466d01 AS
(
  WITH __splink__df_concat_with_tf AS (
    SELECT
      *
    FROM __splink__df_concat_with_tf_8f8a8421b
  ), __splink__df_blocked AS (
    SELECT
      "l"."source_dataset" AS "source_dataset_l",
      "r"."source_dataset" AS "source_dataset_r",
      "l"."id" AS "id_l",
      "r"."id" AS "id_r",
      "l"."PER_FirstName" AS "PER_FirstName_l",
      "r"."PER_FirstName" AS "PER_FirstName_r",
      "l"."tf_PER_FirstName" AS "tf_PER_FirstName_l",
      "r"."tf_PER_FirstName" AS "tf_PER_FirstName_r",
      "l"."PER_LastName" AS "PER_LastName_l",
      "r"."PER_LastName" AS "PER_LastName_r",
      "l"."tf_PER_LastName" AS "tf_PER_LastName_l",
      "r"."tf_PER_LastName" AS "tf_PER_LastName_r",
      "l"."PER_DOB" AS "PER_DOB_l",
      "r"."PER_DOB" AS "PER_DOB_r",
      "l"."PER_SSN" AS "PER_SSN_l",
      "r"."PER_SSN" AS "PE